In [1]:
%env OTEL_ENABLE=1
%env OTEL_EXPORTER_OTLP_ENDPOINT=http://localhost:4317
%matplotlib nbagg

env: OTEL_ENABLE=1
env: OTEL_EXPORTER_OTLP_ENDPOINT=http://localhost:4317


In [2]:
# set this to the host/port where the merlin data server is listening:
MERLIN_DATA_SOCKET = ('127.0.0.1', 6342)
MERLIN_CONTROL_SOCKET = ('127.0.0.1', 6341)
NAV_SHAPE = (128, 128)
SIG_SHAPE = (256, 256)

In [3]:
from matplotlib import pyplot as plt

In [4]:
import time
import os
import numpy as np

from libertem_live.detectors.merlin.acquisition import MerlinAcquisition
from libertem_live.detectors.merlin import MerlinControl
from libertem.executor.pipelined import PipelinedExecutor
from libertem.executor.inline import InlineJobExecutor
from libertem_live.api import LiveContext
from libertem.viz.bqp import BQLive2DPlot
from libertem.udf.sumsigudf import SumSigUDF

In [5]:
from libertem.common.tracing import maybe_setup_tracing

In [6]:
maybe_setup_tracing(service_name="notebook-main")

In [7]:
from libertem.udf import UDF

In [8]:
c = MerlinControl(*MERLIN_CONTROL_SOCKET)

In [9]:
def trigger(aq):
    with c:
        c.cmd('STARTACQUISITION')
    
    with c:
        c.cmd('SOFTTRIGGER')

In [10]:
ctx = LiveContext()

In [11]:
aq = ctx.prepare_acquisition(
    'merlin',
    trigger=trigger,
    nav_shape=NAV_SHAPE,
    sig_shape=SIG_SHAPE,
    host=MERLIN_DATA_SOCKET[0],
    port=MERLIN_DATA_SOCKET[1],
    frames_per_partition=128,
    pool_size=2,
    timeout=5,
)

In [12]:
executor = PipelinedExecutor(n_workers=12)
# executor = InlineJobExecutor()
try:
    print(aq.shape, aq.dtype)
    ctx = LiveContext(executor=executor, plot_class=BQLive2DPlot)
    t0 = time.time()
    ress = [ctx.run_udf(dataset=aq, udf=SumSigUDF(), plots=True)]    
    t1 = time.time()
    print(t1-t0)
finally:
    executor.close()

(128, 128, 256, 256) uint8


Figure(axes=[Axis(label='x', scale=LinearScale(max=1.0, min=0.0)), Axis(label='y', orientation='vertical', sca…

1.315303087234497
